In [ ]:
!pip install pandas_datareader

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import shape
from numpy.lib.type_check import real
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow import keras

crypto='BTC'
currency='USD'

In [2]:
start=dt.datetime(2022,1,1)
end=dt.datetime(2022,7,1)

data=pdr.DataReader(f'{crypto}-{currency}','yahoo',start,end)

In [3]:
data.describe()

,High,Low,Open,Close,Volume,Adj Close
count,183.000000,183.000000,183.000000,183.000000,1.830000e+02,183.000000
mean,37566.494856,35863.056619,36829.955761,36682.233713,2.915230e+10,36682.233713
std,7596.622986,7534.084009,7506.313321,7586.545099,1.066214e+10,7586.545099
min,19371.748047,17708.623047,19010.902344,19017.642578,1.373656e+10,19017.642578
25%,31306.766602,29549.021484,30367.798828,30319.028320,2.192934e+10,30319.028320
50%,39845.925781,38113.664062,39148.449219,39105.148438,2.781953e+10,39105.148438
75%,42943.066406,41399.984375,42266.076172,42247.509766,3.307551e+10,42247.509766
max,48086.835938,47100.437500,47680.925781,47686.812500,8.419661e+10,47686.812500


In [4]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(data['Close'].values.reshape(-1,1))

prediction_days=30
future_day=0

In [5]:
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-01-01,47827.312500,46288.484375,46311.746094,47686.812500,24582667004,47686.812500
2022-01-02,47881.406250,46856.937500,47680.925781,47345.218750,27951569547,47345.218750
2022-01-03,47510.726562,45835.964844,47343.542969,46458.117188,33071628362,46458.117188
2022-01-04,47406.546875,45752.464844,46458.851562,45897.574219,42494677905,45897.574219
2022-01-05,46929.046875,42798.222656,45899.359375,43569.003906,36851084859,43569.003906
...,...,...,...,...,...,...
2022-06-28,21164.423828,20228.812500,20731.544922,20280.634766,21381535161,20280.634766
2022-06-29,20364.156250,19937.791016,20281.169922,20104.023438,23552740328,20104.023438
2022-06-30,20141.160156,18729.656250,20108.312500,19784.726562,26267239923,19784.726562


In [6]:
X=np.array(data.drop('Close',axis=1))

Y=np.array(data['Close'])

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)


In [9]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))


In [15]:
x_train.shape

(122, 5, 1)

In [17]:
y_train.shape

(122,)

In [12]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(loss = 'mean_squared_error',
optimizer = 'adam',
metrics = ['accuracy'])
callback = keras.callbacks.EarlyStopping(monitor='accuracy',
                                            patience=10,
                                            restore_best_weights=True)

In [ ]:
model.fit(x_train,y_train,epochs=30,batch_size=8, callbacks=callback)

In [ ]:
loss, accuracy = model.evaluate((x_val,y_val))
print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
test_start=dt.datetime(2020,1,1)
test_end=dt.datetime.now()

test_data=pdr.DataReader(f'{crypto}-{currency}','yahoo',test_start,test_end)

actual_prices=test_data['Close'].values

total_dataset=pd.concat((data['Close'],test_data['Close']),axis=0)

model_inputs=total_dataset[len(total_dataset)-len(test_data)-prediction_days:].values
model_inputs=model_inputs.reshape(-1,1)

model_inputs=scaler.fit_transform(model_inputs)

x_test=[]

for x in range(prediction_days,len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x,0])

x_test=np.array(x_test)
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))